In [1]:
import pymc as pm

print(f"Running on PyMC v{pm.__version__}")


Running on PyMC v5.23.0


In [2]:

import pytensor as PyT

print(f"Running on PyTensor v{PyT.__version__}")

import pytensor.tensor as pt


Running on PyTensor v2.31.7


In [3]:
basic_model = pm.Model()


In [4]:
import arviz as az
print(f"Running on arviz v{az.__version__}")


Running on arviz v0.22.0


In [ ]:
def extract_summary_dataframe(trace, hdi_prob=0.68):
    """
    Extracts a summary DataFrame from an ArviZ trace object with specified statistics.

    Parameters:
    - trace: ArviZ InferenceData object
    - hdi_prob: float, the probability for the HDI interval (default is 0.68)

    Returns:
    - pandas DataFrame with variables as index and columns: mean, median, sd, hdi_16%, hdi_84%, r_hat
    """
    # Get summary with specified HDI
    summary = az.summary(trace, hdi_prob=hdi_prob)

    # Compute median manually
    medians = trace.posterior.median(dim=["chain", "draw"]).to_dataframe()

    # Merge median into summary
    summary["median"] = medians

    # Reorder and select desired columns
    selected_columns = ['mean', 'median', 'sd', 'hdi_16%', 'hdi_84%', 'r_hat']
    custom_summary_df = summary[selected_columns]

    return custom_summary_df


def sample_until_converged(model, max_attempts=3, rhat_threshold=1.2):
    for attempt in range(1, max_attempts + 1):
        print(f"Sampling attempt {attempt}...")
        
        if attempt == 1:
            trace = pm.sample(draws=2000, tune=1000, chains=4, target_accept=0.9)
        else:
            trace = pm.sample(draws=2000*attempt, tune=1000*attempt, chains=4, target_accept=0.95)
        
        summary = az.summary(trace)
        if (summary['r_hat'] < rhat_threshold).all():
            print(f"Converged on attempt {attempt}")
            return trace
        
        print(f"Attempt {attempt} failed to converge. Retrying...")

    raise RuntimeError("Model did not converge after multiple attempts.")

    
    
def predict_lc_pymc(time_lc,t0,P,rp_rs,a,inc,e,omega,u1,u2,cad):
    oversample = 4
    params = batman.TransitParams()
    params.t0  = t0
    params.per = P
    params.rp  = rp_rs
    params.a   = a
    params.inc = inc
    params.ecc = e
    params.w = omega*180./np.pi
    params.u = [u1,u2]
    params.limb_dark = "quadratic"
        
    cad = pt.switch(cad > 0, cad, 30.)
    
    cad_val = cad if isinstance(cad, float) else 30.  # fallback if needed
    cad_val = cad_val / 24. / 60.

    m = batman.TransitModel(params, time_lc ,supersample_factor = oversample, exp_time = cad/24./60.)

    flux_theo = m.light_curve(params)

    return flux_theo



def pymc_new_general_function(time, flux, unc, t0, other_pars, type_fn, verbose = True, keep_ld_fixed = True, phase_fold = True):

    mask = np.logical_or(np.logical_or(np.isnan(flux),  np.isnan(time)),  np.isnan(unc))
    time = time[~mask]
    unc  = unc[~mask]
    flux = flux[~mask]

    cad = np.min(np.diff(time))*60.*24.
    
    if type_fn == 'Single':
        dur, ab, depth = other_pars
        per = np.max(time)-np.min(time)
        per_unc = 10
        
        indxes = np.where(np.abs(time-t0)<(1.+dur/24.))
        
        use_time = np.array(time)[indxes]
        use_flux = np.array(flux)[indxes]
        use_unc  = np.array(unc)[indxes]
    elif type_fn == 'Periodic':
        per, ab, depth = other_pars
        per_unc = 0.1
        use_time = np.array(time)
        use_flux = np.array(flux)
        use_unc  = np.array(unc)
        
    u1, u2 = ab
        
    with pm.Model() as model:


        time_lc = pm.Data("time_lc", use_time, mutable = False)
        cad = pm.Data("cad", cad, mutable = False)


        t0    = pm.TruncatedNormal("t0", mu=t0, sigma=0.01, lower=t0-0.25, upper=t0+0.25)
        Per   = pm.Normal("Per", mu=per, sigma=per_unc)
        rp_rs = pm.TruncatedNormal("rp_rs", mu=pt.sqrt(depth), sigma=0.01, lower=0, upper=1)
        a_rs  = pm.TruncatedNormal("a_rs", mu=per, sigma=0.1, lower=1)
        cosi  = pm.TruncatedNormal("cosi", mu=0., sigma=0.01, lower=0, upper=1)
        norm  = pm.Normal("norm", mu=1.0, sigma=0.01)

        inc = pm.Deterministic('inclination', pt.arccos(cosi)*180./np.pi)

        if keep_ld_fixed:
            u1 = pm.Data("u1", u1, mutable=False)
            u2 = pm.Data("u2", u2, mutable=False)
        else:
            u1 = pm.TruncatedNormal("u1", mu=u1, sigma=0.01, lower=0, upper=1)
            u2 = pm.TruncatedNormal("u2", mu=u2, sigma=0.01, lower=0, upper=1)
            

        
        #not phase folded
        if phase_fold:
            print('phase fold ingredients: time: ', time_lc, '\nt0: ', t0, '\nPer: ', Per)
            folded_phase = ((time_lc - t0 + 0.5*Per) % Per) -( 0.5*Per)
            print('folded phase', folded_phase)
            sort_indx    = pt.argsort(folded_phase)
            print('sorted phase index', sort_indx)

            phase = folded_phase[sort_indx]
            print('sorted phase', phase)
            p_cad = pt.median(phase[1:] - phase[:-1]) * 60 * 24
            
            p_pars = [phase+t0, p_cad, t0, Per, rp_rs, inc, a_rs, u1, u2]
            p_flux_model = predict_lc_wrapper(p_pars)
        
            pm.Normal("obs", mu=p_flux_model * norm, sigma=use_unc, observed=use_flux[sort_indx])


        
        #phase folded
        else:
            pars = [time_lc, cad, t0, Per, rp_rs, inc, a_rs, u1, u2]

            flux_model = predict_lc_wrapper(pars)

            pm.Normal("obs", mu=flux_model * norm, sigma=use_unc, observed=use+flux)
            
            
            
        #Defining all non-used deterministic variables to be saved later
        b      = pm.Deterministic('b', a_rs*cosi) #
        depth  = pm.Deterministic('depth', rp_rs**2) #
        T_dur0 = (per*24.)/(a*np.pi)
        tau    = pm.Deterministic('tau', rp_rs*T_dur0/pt.sqrt(1.-(b**2.))) #
        dur    = pm.Deterministic('dur', pt.sqrt(1.-(b**2.))*T0+tau ) #
        win    = dur*2 
        if type_fn == 'Periodic':
            intran = np.where(transit_mask(time, per, dur/24, t0))[0]
    #         print('intran', len(np.where(intran)[0]))
        elif type_fn == 'Single':  
            intran = np.abs(time_lc-t0)<(dur/2./24.)
        outran = np.abs(time_lc-t0)>=(dur/24.)
        uq     = np.full(len(flux),np.std(flux[outran]))
        sigs   = np.mean(uq[intran])
                        
        SNR    = pm.Deterministic('SNR', pt.sqrt(len(intran))*depth/sigs ) #
#         SNR    = pm.Deterministic('SNR', pt.sqrt(np.sum((1.-use_flux[intran])/use_unc[intran]))) #

                    

    with model:
        try:
            trace = sample_until_converged(model)
            summary = extract_summary_dataframe(trace)

        except RuntimeError as error:
            return pd.DataFrame(np.nan, index=summary.index, columns=summary.columns)
            

    # Summary statistics
    print(summary)

    if verbose:

        # Trace plots
        az.plot_trace(trace)

        # Posterior plots
        az.plot_posterior(trace)

        # Save results
#         az.to_netcdf(trace, "trace_output.nc")
        plt.show()
    
    return summary
                                


In [11]:

import numpy as np
import batman
import pymc as pm
import pytensor.tensor as pt
from pytensor.graph import Op, Apply


def extract_summary_dataframe(trace, hdi_prob=0.68):
    """
    Extracts a summary DataFrame from an ArviZ trace object with specified statistics.

    Parameters:
    - trace: ArviZ InferenceData object
    - hdi_prob: float, the probability for the HDI interval (default is 0.68)

    Returns:
    - pandas DataFrame with variables as index and columns: mean, median, sd, hdi_16%, hdi_84%, r_hat
    """
    # Get summary with specified HDI
    summary = az.summary(trace, hdi_prob=hdi_prob)

    # Compute median manually
    medians = trace.posterior.median(dim=["chain", "draw"]).to_dataframe()

    # Merge median into summary
    summary["median"] = medians

    # Reorder and select desired columns
    selected_columns = ['mean', 'median', 'sd', 'hdi_16%', 'hdi_84%', 'r_hat']
    custom_summary_df = summary[selected_columns]

    return custom_summary_df


def sample_until_converged(model, max_attempts=3, rhat_threshold=1.2):
    for attempt in range(1, max_attempts + 1):
        print(f"Sampling attempt {attempt}...")
        
        if attempt == 1:
            trace = pm.sample(draws=2000, tune=1000, chains=4, target_accept=0.9, cores =4)
        else:
            trace = pm.sample(draws=2000*attempt, tune=1000*attempt, chains=4, target_accept=0.95, cores =4)
        
        summary = az.summary(trace)
        if (summary['r_hat'] < rhat_threshold).all():
            print(f"Converged on attempt {attempt}")
            return trace
        
        print(f"Attempt {attempt} failed to converge. Retrying...")

    raise RuntimeError("Model did not converge after multiple attempts.")

    

class BatmanOp(Op):
    itypes = [pt.dvector, pt.dscalar, pt.dscalar, pt.dscalar, pt.dscalar, pt.dscalar, pt.dscalar, pt.dscalar, pt.dscalar]
    otypes = [pt.dvector]

    def perform(self, node, inputs, outputs):
        time, t0, per, rp_rs, a_rs, inc, u1, u2, cad = inputs
        params = batman.TransitParams()
        params.t0 = t0
        params.per = per
        params.rp = rp_rs
        params.a = a_rs
        params.inc = inc
        params.ecc = 0.0
        params.w = 90.0
        params.u = [u1, u2,]
        params.limb_dark = "quadratic"
        m = batman.TransitModel(params, time, supersample_factor=4, exp_time=cad/24./60.)
        outputs[0][0] = m.light_curve(params)

    def grad(self, inputs, g_outputs):
        # For now, return zeros (no gradient)
        return [pt.zeros_like(i) for i in inputs]

    
def set_up_variables_for_pymc_fit(time, flux, unc, other_pars, type_fn):
    mask = np.logical_or(np.logical_or(np.isnan(flux),  np.isnan(time)),  np.isnan(unc))
    time = time[~mask]
    unc  = unc[~mask]
    flux = flux[~mask]

    cad = np.min(np.diff(time))*60.*24.
    
    if type_fn == 'Single':
        dur, ab, depth = other_pars
        per = np.max(time)-np.min(time)
        per_unc = 10
        
        indxes = np.where(np.abs(time-t0)<(1.+dur/24.))
        
        use_time = np.array(time)[indxes]
        use_flux = np.array(flux)[indxes]
        use_unc  = np.array(unc)[indxes]
    elif type_fn == 'Periodic':
        per, ab, depth = other_pars
        per_unc = 0.1
        use_time = np.array(time)
        use_flux = np.array(flux)
        use_unc  = np.array(unc)
        
    u1, u2 = ab
        
    return use_time, use_flux, use_unc, per, u1, u2, depth, cad
    
def pymc_new_general_function(time, flux, unc, t0, other_pars, type_fn, verbose = True, keep_ld_fixed = True, phase_fold = True):

    time, flux, unc, per, u1, u2, depth, cad = set_up_variables_for_pymc_fit(time, flux, unc, other_pars, type_fn)
    batman_op = BatmanOp()

    with pm.Model() as model:

        t0    = pm.TruncatedNormal("t0", mu=t0, sigma=0.01, lower=t0-0.25, upper=t0+0.25)
        Per   = pm.Normal("Per", mu=per, sigma=per_unc)
        rp_rs = pm.TruncatedNormal("rp_rs", mu=pt.sqrt(depth), sigma=0.01, lower=0, upper=1)
        a_rs  = pm.TruncatedNormal("a_rs", mu=per, sigma=0.1, lower=1)
        cosi  = pm.TruncatedNormal("cosi", mu=0., sigma=0.01, lower=0, upper=1)
        norm  = pm.Normal("norm", mu=1.0, sigma=0.01)

        inc = pm.Deterministic('inclination', pt.arccos(cosi)*180./np.pi)

        if not keep_ld_fixed:
            u1 = pm.TruncatedNormal("u1", mu=u1, sigma=0.01, lower=0, upper=1)
            u2 = pm.TruncatedNormal("u2", mu=u2, sigma=0.01, lower=0, upper=1)
            
        cad = pt.switch(cad > 0, cad, 30.)


        #not phase folded
        if phase_fold:
            folded_phase = ((time_lc - t0 + 0.5*per) % per) -( 0.5*per)
            sort_indx    = pt.argsort(folded_phase)
            phase = folded_phase[sort_indx]
            p_cad = pt.median(phase[1:] - phase[:-1]) * 60 * 24
            
            p_flux_model = batman_op(phase+t0, t0, per, rp_rs, a_rs, inc, u1, u2, p_cad)
            
            pm.Normal("obs", mu=p_flux_model * norm, sigma=unc, observed=flux[sort_indx])


        
        #phase folded
        else:

            flux_model = batman_op(time,t0, per, rp_rs, a_rs, inc, u1, u2, cad)

            pm.Normal("obs", mu=flux_model * norm, sigma=unc, observed=flux)
            
            
            
        #Defining all non-used deterministic variables to be saved later
        b      = pm.Deterministic('b', a_rs*cosi) #
        depth  = pm.Deterministic('depth', rp_rs**2) #
        T_dur0 = (per*24.)/(a_rs*np.pi)
        tau    = pm.Deterministic('tau', rp_rs*T_dur0/pt.sqrt(1.-(b**2.))) #
        dur    = pm.Deterministic('dur', pt.sqrt(1.-(b**2.))*T0+tau ) #
        win    = dur*2 
        if type_fn == 'Periodic':
            intran = np.where(transit_mask(time, per, dur/24, t0))[0]
    #         print('intran', len(np.where(intran)[0]))
        elif type_fn == 'Single':  
            intran = np.abs(time_lc-t0)<(dur/2./24.)
        outran = np.abs(time_lc-t0)>=(dur/24.)
        uq     = np.full(len(flux),np.std(flux[outran]))
        sigs   = np.mean(uq[intran])
                        
        SNR    = pm.Deterministic('SNR', pt.sqrt(len(intran))*depth/sigs ) #
#         SNR    = pm.Deterministic('SNR', pt.sqrt(np.sum((1.-use_flux[intran])/use_unc[intran]))) #

                    

    with model:
        try:

            trace = sample_until_converged(model)
            summary = extract_summary_dataframe(trace)

        except RuntimeError as error:
            return pd.DataFrame(np.nan, index=summary.index, columns=summary.columns)
            

    # Summary statistics
    print(summary)

    if verbose:

        # Trace plots
        az.plot_trace(trace)

        # Posterior plots
        az.plot_posterior(trace)

        # Save results
#         az.to_netcdf(trace, "trace_output.nc")
        plt.show()
    
    return summary


NameError: name 'time' is not defined

In [ ]:
def predict_lc(time_lc,t0,P,rp_rs,cosi,a,u1,u2,cad):
    oversample = 4
    e = 0.
    omega = np.pi/2.
    inc = np.arccos(cosi)*180./np.pi
    params = batman.TransitParams()
    params.t0  = t0
    params.per = P
    params.rp  = rp_rs
    params.a   = a
    params.inc = inc
    params.ecc = e
    params.w = omega*180./np.pi
    params.u = [u1,u2]
    params.limb_dark = "quadratic"
        
    if not cad>0:
        cad = 30.
    m = batman.TransitModel(params, time_lc ,supersample_factor = oversample, exp_time = cad/24./60.)

    flux_theo = m.light_curve(params)

    return flux_theo

def predict_lc_wrapper(pars):
    time_lc, cad, t0, P, rp_rs, cosi, a, u1, u2, cad = pars
    return predict_lc(time_lc, t0, P, rp_rs, cosi, a, u1, u2, cad)

def lnprob_MCMC_global(flux,unc,time,cad,tc, u1, u2, pars):
#     flux,unc,time,cad,tc, u1, u2 = PARAMS
    t0 = pars[0]
    P = pars[1]
    rp_rs = pars[2]
    cosi = pars[3]
    a = pars[4]
    norm = pars[5]
    b = np.abs(a*cosi)
    #make sure all pars are good

    if np.abs(tc-t0)>0.25: return -np.inf
    if np.max([u1,u2,cosi,rp_rs]) > 1.: return -np.inf
    if np.min([u1,u2,cosi,rp_rs]) < 0.: return -np.inf
    if a < 1: return -np.inf
    if b > 1: return -np.inf

    # flux_theo = predict_lc(time,t0,P,rp_rs,cosi,a,u1,u2,cad)

    x = ((time - t0 + 0.5*P) % P) -( 0.5*P)

    flux_theo = predict_lc(x+t0,t0,P,rp_rs,cosi,a,u1,u2,cad)

    m = np.abs(x) < 0.5

    flux=flux*norm
    unc=unc*norm

    flux_theo_phase, flux_phase, unc_phase = easy_data_phaseFold(np.array(m), flux_theo, flux, unc)
    
    result = 0.-0.5*np.sum(((flux_theo_phase-flux_phase)/unc_phase)**2.)
    if np.isfinite(result) != True:
        #print('bad result')
        return -np.inf
    return result


def lnprob_MCMC_global_single(flux,unc,time,cad,tc, u1, u2, pars):
#     flux,unc,time,per,cad,tc, u1, u2= PARAMS
    t0 = pars[0]
    rp = pars[1]
    cosi = pars[2]
    a = pars[3]
    norm = pars[4]
    b = np.abs(a*cosi)
    #make sure all pars are good

    if np.abs(tc-t0)>0.25: return -np.inf
#     if np.max(rp)>0.25: return -np.inf #attempting to set some upper bounds here just for the singles
    if np.max([u1,u2,cosi,rp]) > 1.: return -np.inf
    if np.min([u1,u2,cosi,rp]) < 0.: return -np.inf
    if a < 1: return -np.inf
    if b > 1: return -np.inf

    flux_theo = predict_lc(time,t0,per,rp,cosi,a,u1,u2,cad)
    flux=flux*norm
    unc=unc*norm
    result = 0.-0.5*np.sum(((flux_theo-flux)/unc)**2.)
    if np.isfinite(result) != True:
        #print('bad result')
        return -np.inf
    return result


